# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

In addition to implementing code, there is a brief writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) that can be used to guide the writing process. Completing both the code in the Ipython notebook and the writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/322/view) for this project.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note: If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".**

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

**Run the cell below to import some packages.  If you get an `import error` for a package you've already installed, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt.  Also, see [this forum post](https://carnd-forums.udacity.com/cq/viewquestion.action?spaceKey=CAR&id=29496372&questionTitle=finding-lanes---import-cv2-fails-even-though-python-in-the-terminal-window-has-no-problem-with-import-cv2) for more troubleshooting tips.**  

## Import Packages

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

## Read in an Image

In [ ]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')

#printing out some stats and plotting
print('This image is:', type(image), 'with dimesions:', image.shape)
plt.imshow(image)  # if you wanted to show a single color channel image called 'gray', for example, call as plt.imshow(gray, cmap='gray')

## Ideas for Lane Detection Pipeline

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

## Helper Functions

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [ ]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def clear_globals():
    global all_left_x1s, all_left_x2s, all_right_x1s, all_right_x2s
    all_left_x1s = []
    all_left_x2s = []
    all_right_x1s = []
    all_right_x2s = []

def draw_lines(img, lines, horizon=0, bottom=0, prune=False, extrapolate=False, average_frames=False, color=[255, 0, 0], thickness=2, debug=False):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    
    # some defaults if not specified
    if horizon == 0:
        horizon = int(img.shape[0]/2)
    if bottom == 0:
        bottom = img.shape[0]
    if debug:
        print("horizon = {0}\nbottom = {1}".format(horizon,bottom))
        
    # define local lists, used to collect all extended Hough lines per side
    left_x1s = []
    left_x2s = []
    left_weights = []
    right_x1s = []
    right_x2s = []
    right_weights = []
    
    # define global lists, used to average extended Hough lines across frames
    if average_frames and 'all_left_x1s' not in globals():
        clear_globals()
        
    for line in lines:
        for x1,y1,x2,y2 in line:
            # calculate some interesting parameters for each line y=mx+b
            m, b = np.polyfit((x1,x2),(y1,y2),1)
            line_len = int(np.sqrt((x2-x1)**2 + (y2-y1)**2))
            xhorizon = int((horizon-b)/m)
            xbottom = int((bottom-b)/m)
            
            # prune lines with unexpected slopes if desired
            if prune:
                ma = abs(m)
                if debug:
                    print("({0},{1})->({2},{3}) m={4} b={5}".format(x1,y1,x2,y2,m,b))
                if ma < 0.2 or ma > 0.8:
                    continue
                    
            if not extrapolate:
                # just add every line if not extrapolating
                cv2.line(img, (x1, y1), (x2, y2), color, thickness)
            else:
                # find left & right hand lanes
                xcenter = img.shape[1]/2
                if x1 < xcenter and x2 < xcenter:
                    # left hand lane
                    left_x1s.append(xbottom)
                    left_x2s.append(xhorizon)
                    left_weights.append(line_len)
                    if debug:
                        print("L: ({0},{1})->({2},{3}) w={4}".format(xbottom,bottom,xhorizon,horizon,line_len))
                elif x1 > xcenter and x2 > xcenter:
                    # right hand lane
                    right_x1s.append(xbottom)
                    right_x2s.append(xhorizon)
                    right_weights.append(line_len)
                    if debug:
                        print("R: ({0},{1})->({2},{3}) w={4}".format(xbottom,bottom,xhorizon,horizon,line_len))
                else:
                    # why is it crossing over?
                    print("ERROR: line crossing sides: ({0},{1}) -> ({2},{3})".format(x1,y1,x2,y2))
                    continue
                
    if extrapolate:
        # draw left hand extrapolated lane
        if len(left_weights) > 0:  # sometimes, Hough gives me no length - weird
            x1avg = int(sum(x1*weight for x1,weight in zip(left_x1s,left_weights))/sum(left_weights))
            x2avg = int(sum(x2*weight for x2,weight in zip(left_x2s,left_weights))/sum(left_weights))
            if average_frames:
                all_left_x1s.append(x1avg)
                all_left_x2s.append(x2avg)
                frame_weights = [10,10,10,5,5,5,2,2,2]   # current frame + previous 8 frames
                x1frameavg = int(sum(x1*weight for x1,weight in zip(reversed(all_left_x1s[-9:]),frame_weights))/sum(frame_weights[:len(all_left_x1s)]))
                x2frameavg = int(sum(x2*weight for x2,weight in zip(reversed(all_left_x2s[-9:]),frame_weights))/sum(frame_weights[:len(all_left_x2s)]))
            else:
                x1frameavg,x2frameavg = x1avg,x2avg
            if debug:
                print("L_avg: ({0},{1})->({2},{3})".format(x1avg,bottom,x2avg,horizon))
                print("LF_avg: ({0},{1})->({2},{3})".format(x1frameavg,bottom,x2frameavg,horizon))
            cv2.line(img, (x1frameavg, bottom), (x2frameavg, horizon), color, thickness)
        elif average_frames:   # just out and out cheating by copying last frame if we lose one
            print("WARNING: no left lanes in frame {0}".format(len(all_left_x1s)-1))
            cv2.line(img, (all_left_x1s[-1], bottom), (all_left_x2s[-1], horizon), color, thickness)
        # draw right hand extrapolated lane
        if len(right_weights) > 0:  # sometimes, Hough gives me no length - weird
            x1avg = int(sum(x1*weight for x1,weight in zip(right_x1s,right_weights))/sum(right_weights))
            x2avg = int(sum(x2*weight for x2,weight in zip(right_x2s,right_weights))/sum(right_weights))
            if average_frames:
                all_right_x1s.append(x1avg)
                all_right_x2s.append(x2avg)
                frame_weights = [10,10,10,5,5,5,2,2,2]   # current frame + previous 8 frames
                x1frameavg = int(sum(x1*weight for x1,weight in zip(reversed(all_right_x1s[-9:]),frame_weights))/sum(frame_weights[:len(all_right_x1s)]))
                x2frameavg = int(sum(x2*weight for x2,weight in zip(reversed(all_right_x2s[-9:]),frame_weights))/sum(frame_weights[:len(all_right_x2s)]))
            else:
                x1frameavg,x2frameavg = x1avg,x2avg
            if debug:
                print("R_avg: ({0},{1})->({2},{3})".format(x1avg,bottom,x2avg,horizon))
                print("RF_avg: ({0},{1})->({2},{3})".format(x1frameavg,bottom,x2frameavg,horizon))
            cv2.line(img, (x1frameavg, bottom), (x2frameavg, horizon), color, thickness)
        elif average_frames:   # just out and out cheating by copying last frame if we lose one
            print("WARNING: no right lanes in frame {0}".format(len(all_right_x1s)-1))
            cv2.line(img, (all_right_x1s[-1], bottom), (all_right_x2s[-1], horizon), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    if lines is not None: # test for empty Hough lines
        draw_lines(line_img, lines, thickness=10)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

def show_image(location, title, img):
    plt.subplot(*location)
    plt.title(title)
    if len(img.shape) == 3:
        plt.imshow(img)
    else:
        plt.imshow(img, cmap='gray')

## Test Images

Build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [ ]:
import os
os.listdir("test_images/")
# create output directory
if not os.path.exists("test_output_images/"):
    os.makedirs("test_output_images/")

## Build a Lane Finding Pipeline



Build the pipeline and run your solution on all test_images. Make copies into the test_images directory, and you can use the images in your writeup report.

Try tuning the various parameters, especially the low and high Canny thresholds as well as the Hough lines parameters.

In [ ]:
# TODO: Build your pipeline that will draw lane lines on the test_images
# then save them to the test_images directory.

def find_lane_lines(img,debug=False,video=False,show_hough=False,print_pics=False,detect_lane_colors=True,debug_name=""):
    if debug:
        print("initial: {0}".format(img.shape))

    # if color & RGBA
    if len(img.shape) > 2 and img.shape[2] == 4:
        # convert the image from RGBA2RGB
        img = cv2.cvtColor(img, cv2.COLOR_RGBA2RGB)
        if debug:
            print("RGBA->RGB: {0}".format(img.shape))

    if detect_lane_colors:
        # convert to HSV
        hsv = cv2.cvtColor(img,cv2.COLOR_RGB2HSV)
    
        # select white pixels
        lower_white = np.array([0,0,180], dtype=np.uint8) # HSV (0-180, 0-255, 0-255)
        upper_white = np.array([255,25,255], dtype=np.uint8) #HSV
        white = cv2.inRange(hsv,lower_white,upper_white)
        # select yellow pixels (traffic yellow is 50/360 degrees, 100% sat, 98% value)
        lower_yellow = np.array([18,120,150], dtype=np.uint8) # HSV
        upper_yellow = np.array([30,255,255], dtype=np.uint8) # HSV
        yellow = cv2.inRange(hsv,lower_yellow,upper_yellow)
        
        color_mask = cv2.bitwise_or(white,yellow) # could return this as gray, but...
        gray_img = grayscale(img)
        # instead, let's filter out some things we don't care about
        # by only taking bright lanes markers
        # who knows how it'll work in the rain or dark
        gray_mask = cv2.bitwise_and(gray_img,color_mask)
        retval,gray = cv2.threshold(gray_mask,180,255,cv2.THRESH_BINARY)
        ### why doesn't this work???
        ### gray = cv2.inRange(gray_mask,180,255)
        # define our parameters for Canny
        low_threshold = 180   # remember, we already filtered out anything below 180
        high_threshold = 240  # everything's so shiny and bright in this grayscale!!!
    else:
        # grayscale the image
        gray = grayscale(img)
        # define our parameters for Canny
        low_threshold = 70
        high_threshold = 210

    if debug:
        print("gray: {0}".format(img.shape))

    # define a kernel size and apply Gaussian smoothing
    kernel_size = 3
    blur = gaussian_blur(gray, kernel_size)
        
    # apply Canny
    edges = canny(blur, low_threshold, high_threshold)
        
    # next we'll create a masked edges image
    # this time we are defining a four sided polygon to mask
    img_height, img_width = img.shape[0:2]
    center = (int(img_width/2),int(img_height/2))
    # ideally, this would be a visually detected horizon,
    # but for now, just somewhere below the center
    horizon_line = int(center[1]*1.25)
    # ideally, this would visually detect and exclude the hoodline,
    # but for now, just the bottom of the frame
    bottom_line = img_height
    # ideally, these would be based on expected lane widths at each y coordinate,
    # but for now, just use some percentages based on frame size
    bottom_left_x = int(img_width*0.10)
    bottom_right_x = int(img_width*0.93)
    horizon_left_x = int(center[0]*0.9)
    horizon_right_x = int(center[0]*1.1)
    vertices = np.array([[(bottom_left_x, bottom_line),
                          (horizon_left_x, horizon_line),
                          (horizon_right_x, horizon_line),
                          (bottom_right_x, bottom_line)]], dtype=np.int32)
    masked_edges = region_of_interest(edges, vertices)
        
    # define the Hough transform parameters
    rho = 1
    theta = np.pi/180
    threshold = 20              # somewhat high threshold so we get rid of Bott's dots
    min_line_length = 15        # has to be small to pick up dashed lines further out
    max_line_gap = 20           # this is somewhat high, mainly to pick up lanes with bad lighting further out
    
    # run Hough on edge detected image and draw lines on the blank
    #lines = hough_lines(masked_edges, rho, theta, threshold, min_line_length, max_line_gap)
    hough = cv2.HoughLinesP(masked_edges, rho, theta, threshold, np.array([]), minLineLength=min_line_length, maxLineGap=max_line_gap)
    hlines = np.zeros((masked_edges.shape[0], masked_edges.shape[1], 3), dtype=np.uint8)
    xlines = np.zeros((masked_edges.shape[0], masked_edges.shape[1], 3), dtype=np.uint8)
    if hough is not None: # test for empty Hough lines
        draw_lines(xlines, hough, extrapolate=True, average_frames=video,
                   horizon=horizon_line, bottom=bottom_line, prune=True,
                   thickness=10, debug=debug)
        if show_hough:
            draw_lines(hlines, hough, color=[0,0,255], prune=True, thickness=12, debug=debug)


    # draw the lines on the edge image
    combo = weighted_img(img, xlines)
    if show_hough:
        combo = weighted_img(combo, hlines)
    
    if print_pics:
        # display all initial, intermediate, and final pictures in a grid
        plt.figure(figsize=[16, 5])
        for i, imgname in enumerate(['img', 'gray', 'masked_edges', 'hlines', 'xlines', 'combo']):
            show_image((2, 3, i+1), debug_name+" "+imgname, eval(imgname))
        
    return combo

# go through all test_images
clear_globals()
for file in os.scandir("test_images"):
    if not file.name.startswith('.') and file.is_file():
        print("processing {0}...".format(file.name))

        # read in the image
        image = mpimg.imread(file.path)
        
        # take it through the pipeline
        output = find_lane_lines(image,debug=False,show_hough=False,print_pics=False,debug_name=file.name)
        
        # write the resulting image
        outfile = 'test_output_images/output_' + file.name
        mpimg.imsave(outfile,output)
       

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

**Note: if you get an `import error` when you run the next cell, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt. Also, check out [this forum post](https://carnd-forums.udacity.com/questions/22677062/answers/22677109) for more troubleshooting tips.**

**If you get an error that looks like this:**
```
NeedDownloadError: Need ffmpeg exe. 
You can download it by calling: 
imageio.plugins.ffmpeg.download()
```
**Follow the instructions in the error message and check out [this forum post](https://carnd-forums.udacity.com/display/CAR/questions/26218840/import-videofileclip-error) for more troubleshooting tips across operating systems.**

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)

    result = find_lane_lines(image,show_hough=False,debug=False,video=True)
    
    return result

Let's try the one with the solid white lane on the right first ...

In [ ]:
clear_globals()
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

## Improve the draw_lines() function

**At this point, if you were successful with making the pipeline and tuning parameters, you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform. As mentioned previously, try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines. You can see an example of the result you're going for in the video "P1_example.mp4".**

**Go back and modify your draw_lines function accordingly and try re-running your pipeline. The new output should draw a single, solid line over the left lane line and a single, solid line over the right lane line. The lines should start from the bottom of the image and extend out to the top of the region of interest.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
clear_globals()
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Writeup and Submission

If you're satisfied with your video outputs, it's time to make the report writeup in a pdf or markdown file. Once you have this Ipython notebook ready along with the writeup, it's time to submit for review! Here is a [link](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) to the writeup template file.


My pipeline is essentially the same as the one in the lessons, with the only big change being that I replaced the initial grayscale image before the Canny edge detection with a color selection scheme to help weed out more false positives. This allowed me to simplify the Canny edge detector and Hough line segment detector parameters to get reasonable results though there's still room for improvement here. Specifically, there are three frames across the three videos that don't have lanes detected on one side or the other and so I have to hack around it by leveraging the inter-frame average. I display warnings for those frames.

For the post-Hough line filtering, I looked at the lane lengths and used those as weights within each frame to decide which lines to extend, the idea being the longer lines right in front of the car are more reliable indicators of the current direction than something further out. And, then, to reduce jitter from frame to frame, I do a weighted average of the current frame with the last 8 frames, if they exist.

I decided to take on the challenge from the very outset and so I created still pictures of some useful scenes from the video and copied them into test_images. Realizing that the extents of the challenge pictures were different than the regular images, I knew I had to come up with a configurable region of interest (ROI). The only oddball thing was that the solidYellowLeft.mp4 video starts out with the left yellow lane much closer to the left edge of the frame than in the still images so calibrating the percentages to the stills didn't work initially and I had to expand the bottom part of the polygon to address the beginning of the video. There's clearly room for improvement in the technique.

Staying with the ROI, I currently don't crop out the hoodline that's in the bottom of the frames in the challenge video. Doing some sort of average lane length and car edge detection to dynamically calculate the ROI would be useful. Even better would be a camera calibration operation before we start and possibly every once in a while if this is not a well mounted camera. This calibration could even correct for pitch and roll of the camera and rotate and/or crop the image appropriately. The same thing applies to finding the horizon line. Shouldn't be too hard, if I knew the right vision techniques, to find it automatically using OpenCV.

Beyond those items, where this algorithm can go wrong is with tight curves. To fix this, instead of using Hough to find lines, we'd need to somehow find splines to fit the lanes. This algorithm will also fail with any lane changes or white or yellow car changing lanes right in front of us (i.e., under the horizon). This may be fixable with a larger frame-to-frame average. Of course, to fix all of this the right way, one can imagine the ROI calculation, the Canny and Hough parameters, and the intra- and inter-frame averaging can be done much better with ML algorithms to improve them, but I'm sure that's what this class is about.

One last comment about my code is that there are a lot of performanc issues. For one, the "frame rate" on my 12" MacBook is about 20it/s which is just horrendous given I'm only detecing lane lines. This could be due to a variety of factors including running inside Jupyter, but I know this needs to be sped up by a factor of 10x or more. And then one thing I need to do is clean up the globals. Right now, a long video woudl just pile up and I really only need to keep the last 8 frames worth of line data but I didn't get around to doing that.

Finally, in terms of the work required to do this, the algorithm itself was relatively easy and I think I had figured it out within 5-10 minutes of looking at this notebook. It then took me over 20 hours of work to get the algorithm implemented and tested. Most of this was fighting with setup issues, Python and library syntax, data types, image format issues, etc. The parameter tweaking probably only took me an hour in total. Hope future projects go easier than this one. What a learning experience for the first week.


## Submission

If you're satisfied with your video outputs it's time to submit!  Submit this ipython notebook for review.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
clear_globals()
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))